In [1]:
import pandas as pd
import numpy as np

In [65]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./data/data.csv')

C:\Users\zyygf\AppData\Local\Temp\ipykernel_63484\269992028.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/data.csv')


In [3]:
useful_columns = ['match_id', 'point_id', 'skill', 'evaluation_code', 'attack_code', 'serving_team', 'player_name', 'player_id', 'team', 'point', 'home_team_score', 'visiting_team_score']

In [4]:
temp_df = df[useful_columns]
temp_df

,match_id,point_id,skill,evaluation_code,attack_code,serving_team,player_name,player_id,team,point,home_team_score,visiting_team_score
0,5d7a5827529932230e12cc7d0fcbee40,0,NaN,NaN,NaN,University of Iowa,NaN,NaN,University of Iowa,False,0,1
1,5d7a5827529932230e12cc7d0fcbee40,0,NaN,NaN,NaN,University of Iowa,NaN,NaN,University of Iowa,False,0,1
2,5d7a5827529932230e12cc7d0fcbee40,0,NaN,NaN,NaN,University of Iowa,NaN,NaN,Ohio State University,False,0,1
3,5d7a5827529932230e12cc7d0fcbee40,0,NaN,NaN,NaN,University of Iowa,NaN,NaN,Ohio State University,False,0,1
4,5d7a5827529932230e12cc7d0fcbee40,0,Serve,-,NaN,University of Iowa,Addie VanderWeide,-334910.0,University of Iowa,False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1328919,0e0cd40b7c51f99cb1f6fa00e8021289,139,Set,#,NaN,University of Oklahoma,Callie Williams,-394347.0,Texas Christian University,False,21,25
1328920,0e0cd40b7c51f99cb1f6fa00e8021289,139,Attack,#,X5,University of Oklahoma,Julia Adams,-277767.0,Texas Christian University,False,21,25
1328921,0e0cd40b7c51f99cb1f6fa00e8021289,139,Dig,=,NaN,University of Oklahoma,Taylor Preston,-382320.0,University of Oklahoma,False,21,25
1328922,0e0cd40b7c51f99cb1f6fa00e8021289,139,NaN,NaN,NaN,University of Oklahoma,NaN,NaN,Texas Christian University,True,21,25


In [5]:
temp_df['side'] = temp_df.apply(lambda x: 'S' if x['serving_team'] == x['team'] else 'R' if pd.notna(x['team']) else None, axis=1)
temp_df

C:\Users\zyygf\AppData\Local\Temp\ipykernel_63484\4159250422.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['side'] = temp_df.apply(lambda x: 'S' if x['serving_team'] == x['team'] else 'R' if pd.notna(x['team']) else None, axis=1)


,match_id,point_id,skill,evaluation_code,attack_code,serving_team,player_name,player_id,team,point,home_team_score,visiting_team_score,side
0,5d7a5827529932230e12cc7d0fcbee40,0,NaN,NaN,NaN,University of Iowa,NaN,NaN,University of Iowa,False,0,1,S
1,5d7a5827529932230e12cc7d0fcbee40,0,NaN,NaN,NaN,University of Iowa,NaN,NaN,University of Iowa,False,0,1,S
2,5d7a5827529932230e12cc7d0fcbee40,0,NaN,NaN,NaN,University of Iowa,NaN,NaN,Ohio State University,False,0,1,R
3,5d7a5827529932230e12cc7d0fcbee40,0,NaN,NaN,NaN,University of Iowa,NaN,NaN,Ohio State University,False,0,1,R
4,5d7a5827529932230e12cc7d0fcbee40,0,Serve,-,NaN,University of Iowa,Addie VanderWeide,-334910.0,University of Iowa,False,0,1,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1328919,0e0cd40b7c51f99cb1f6fa00e8021289,139,Set,#,NaN,University of Oklahoma,Callie Williams,-394347.0,Texas Christian University,False,21,25,R
1328920,0e0cd40b7c51f99cb1f6fa00e8021289,139,Attack,#,X5,University of Oklahoma,Julia Adams,-277767.0,Texas Christian University,False,21,25,R
1328921,0e0cd40b7c51f99cb1f6fa00e8021289,139,Dig,=,NaN,University of Oklahoma,Taylor Preston,-382320.0,University of Oklahoma,False,21,25,S
1328922,0e0cd40b7c51f99cb1f6fa00e8021289,139,NaN,NaN,NaN,University of Oklahoma,NaN,NaN,Texas Christian University,True,21,25,R


In [6]:
temp_df.loc[temp_df['point'] == True, ['skill', 'evaluation_code']] = ['Point', '']

In [7]:
skill_code_dict = {
    'Serve': 'SV',
    'Reception': 'R', 
    'Set': 'S', 
    'Attack': 'A', 
    'Dig': 'D', 
    'Point': 'P', 
    'Block': 'B', 
    'Freeball': 'F'
}
temp_df['skill_code'] = temp_df['skill'].map(skill_code_dict)

C:\Users\zyygf\AppData\Local\Temp\ipykernel_63484\726817799.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['skill_code'] = temp_df['skill'].map(skill_code_dict)


In [8]:
temp_df.loc[temp_df['skill_code'].isna(), 'side'] = np.nan
temp_df.loc[(temp_df['attack_code'].isna()) & (temp_df['skill_code'] == 'A'), 'attack_code'] = 'NA'

In [9]:
def create_short_state_code(df):
    # Initialize the 'short_state' and 'prev_state' columns with NaN values
    df['short_state'] = np.nan
    df['prev_state'] = np.nan
    
    # Variable to store the previous state
    previous_state = None
    
    # Iterate over the dataframe row by row
    for i in range(len(df)):
        skill_code = df.at[i, 'skill_code']
        side = df.at[i, 'side']
        evaluation_code = df.at[i, 'evaluation_code']
        attack_code = df.at[i, 'attack_code']
        
        # If skill_code is NaN, this is the separator between points
        if pd.isna(skill_code):
            previous_state = None
            continue

        # Rule 2: If the skill_code is 'SV' (serve), start a new state code with 'S_SV'
        if skill_code == 'SV':
            df.at[i, 'prev_state'] = previous_state  # Store previous state
            df.at[i, 'short_state'] = f"{side}_SV"
            previous_state = df.at[i, 'short_state']
            continue

        # For attacks, append attack_code, no evaluation_code
        if skill_code == 'A':
            df.at[i, 'prev_state'] = previous_state  # Store previous state
            df.at[i, 'short_state'] = f"{side}_A{attack_code}"
            previous_state = df.at[i, 'short_state']
            continue

        # For other skills, append evaluation_code
        df.at[i, 'prev_state'] = previous_state  # Store previous state
        df.at[i, 'short_state'] = f"{side}_{skill_code}{evaluation_code}"
        previous_state = df.at[i, 'short_state']

    return df

In [10]:
def remove_consecutive_nan_rows(df):
    # Identify rows where all values are NaN
    nan_rows = df['skill_code'].isna()

    # Create a mask to mark the start of each block of consecutive NaNs
    drop_mask = nan_rows & nan_rows.shift(fill_value=False)

    # Remove consecutive NaN rows but keep the first one
    df_cleaned = df[~drop_mask].copy()

    return df_cleaned

# Apply the function to clean the DataFrame
temp_df = remove_consecutive_nan_rows(temp_df)
temp_df = temp_df.reset_index(drop=True)

In [11]:
state_df = create_short_state_code(temp_df)
state_df

C:\Users\zyygf\AppData\Local\Temp\ipykernel_63484\3885978384.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'S_SV' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'short_state'] = f"{side}_SV"
C:\Users\zyygf\AppData\Local\Temp\ipykernel_63484\3885978384.py:36: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'S_SV' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[i, 'prev_state'] = previous_state  # Store previous state


,match_id,point_id,skill,evaluation_code,attack_code,serving_team,player_name,player_id,team,point,home_team_score,visiting_team_score,side,skill_code,short_state,prev_state
0,5d7a5827529932230e12cc7d0fcbee40,0,NaN,NaN,NaN,University of Iowa,NaN,NaN,University of Iowa,False,0,1,NaN,NaN,NaN,NaN
1,5d7a5827529932230e12cc7d0fcbee40,0,Serve,-,NaN,University of Iowa,Addie VanderWeide,-334910.0,University of Iowa,False,0,1,S,SV,S_SV,NaN
2,5d7a5827529932230e12cc7d0fcbee40,0,Reception,+,NaN,University of Iowa,Gabby Gonzales,-278794.0,Ohio State University,False,0,1,R,R,R_R+,S_SV
3,5d7a5827529932230e12cc7d0fcbee40,0,Set,#,NaN,University of Iowa,Mac Podraza,-262192.0,Ohio State University,False,0,1,R,S,R_S#,R_R+
4,5d7a5827529932230e12cc7d0fcbee40,0,Attack,#,X6,University of Iowa,Emily Londot,-308957.0,Ohio State University,False,0,1,R,A,R_AX6,R_S#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
953854,0e0cd40b7c51f99cb1f6fa00e8021289,139,Set,#,NaN,University of Oklahoma,Callie Williams,-394347.0,Texas Christian University,False,21,25,R,S,R_S#,R_R#
953855,0e0cd40b7c51f99cb1f6fa00e8021289,139,Attack,#,X5,University of Oklahoma,Julia Adams,-277767.0,Texas Christian University,False,21,25,R,A,R_AX5,R_S#
953856,0e0cd40b7c51f99cb1f6fa00e8021289,139,Dig,=,NaN,University of Oklahoma,Taylor Preston,-382320.0,University of Oklahoma,False,21,25,S,D,S_D=,R_AX5
953857,0e0cd40b7c51f99cb1f6fa00e8021289,139,Point,,NaN,University of Oklahoma,NaN,NaN,Texas Christian University,True,21,25,R,P,R_P,S_D=


In [24]:
# Drop two error rows
state_df = state_df.drop([786697, 786698]).reset_index(drop=True)

In [31]:
def create_transition_matrix(df, state_column, prev_state_column):
    # Filter rows where both 'prev_state' and 'state' are not NaN
    valid_transitions = df.dropna(subset=[state_column])

    # Count occurrences of each transition (prev_state -> state)
    transition_counts = valid_transitions.groupby([prev_state_column, state_column]).size().unstack(fill_value=0)

    # Get the full set of unique states from both indices (prev_state) and columns (state)
    all_states = sorted(set(transition_counts.index).union(set(transition_counts.columns)))

    # Reindex the matrix to ensure all states are present in both rows and columns
    transition_matrix = transition_counts.reindex(index=all_states, columns=all_states, fill_value=0)

    # Normalize counts to get probabilities (empirical transition matrix)
    transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0).fillna(0)

    return transition_matrix

# Apply the function to the dataframe
transition_matrix = create_transition_matrix(state_df, 'short_state', 'prev_state')

In [32]:
# Set transition probabilities between absorbing states to 1
transition_matrix.at['S_P', 'S_P'] = 1
transition_matrix.at['R_P', 'R_P'] = 1
transition_matrix

short_state,R_ACB,R_ACD,R_ACF,R_ANA,R_APP,R_AV0,R_AV5,R_AV6,R_AV8,R_AVB,...,S_F/,S_F=,S_P,S_R-,S_R/,S_R=,S_S#,S_S-,S_S=,S_SV
prev_state,,,,,,,,,,,,,,,,,,,,,
R_ACB,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.091298,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
R_ACD,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.060606,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
R_ACF,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.085981,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
R_ANA,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000181,0.000181,0.090054,0.000181,0.000000,0.000000,0.000362,0.000000,0.0,0.0
R_APP,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000541,0.000000,0.000000,0.000000,0.000541,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S_R=,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
S_S#,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
S_S-,0.0,0.0,0.0,0.089571,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000818,0.048671,0.010634,0.000000,0.000000,0.000000,0.000818,0.000818,0.0,0.0


In [37]:
# transition_matrix.to_csv('./data/simplified_transition_matrix.csv', sep=';')
transition_matrix.to_csv('./data/simplified_transition_matrix.csv')

#### Symmetric avg

In [38]:
sym_transition_matrix = transition_matrix.copy()

# Function to extract side and skill code from a state (e.g., 'R_A' -> ('R', 'A'))
def extract_side_and_skill(state):
    side, skill_code = state.split('_')
    return side, skill_code

# Apply the function to extract sides and skill codes for both the index (prev_state) and columns (state)
prev_sides, prev_skills = zip(*sym_transition_matrix.index.map(extract_side_and_skill))
state_sides, state_skills = zip(*sym_transition_matrix.columns.map(extract_side_and_skill))

# Convert the sides and skill codes into lists for easier comparison
prev_sides = list(prev_sides)
prev_skills = list(prev_skills)
state_sides = list(state_sides)
state_skills = list(state_skills)

# Set to track processed pairs to avoid duplicated searches
processed_pairs = set()

# Iterate over the transition matrix to apply the symmetric average
for i, P1_prev_state in enumerate(sym_transition_matrix.index):
    for j, P1_state in enumerate(sym_transition_matrix.columns):
        
        # Extract side and skill code for P1_prev_state and P1_state
        P1_prev_side, P1_prev_skill = prev_sides[i], prev_skills[i]
        P1_side, P1_skill = state_sides[j], state_skills[j]

        # Skip if this pair has already been processed (symmetric search optimization)
        if (P1_prev_state, P1_state) in processed_pairs:
            continue

        # Condition 1: Match 'R' to 'R' (prev_state and state have the same skill codes)
        if P1_prev_side == 'R' and P1_side == 'R':
            # Construct P2 by changing the sides from 'R' to 'S'
            P2_prev_state = f"S_{P1_prev_skill}"
            P2_state = f"S_{P1_skill}"
            
            # Check if P2 exists in the transition matrix
            if P2_prev_state in sym_transition_matrix.index and P2_state in sym_transition_matrix.columns:
                # Calculate symmetric average for condition 1
                prob1 = sym_transition_matrix.at[P1_prev_state, P1_state]
                prob2 = sym_transition_matrix.at[P2_prev_state, P2_state]
                avg_prob = (prob1 + prob2) / 2

                # Update the transition probabilities with the average
                sym_transition_matrix.at[P1_prev_state, P1_state] = avg_prob
                sym_transition_matrix.at[P2_prev_state, P2_state] = avg_prob

                # Mark this pair as processed
                processed_pairs.add((P1_prev_state, P1_state))
                processed_pairs.add((P2_prev_state, P2_state))

        # Condition 2: Match 'R' to 'S' (prev_state and state swap sides with same skill codes)
        if P1_prev_side == 'R' and P1_side == 'S':
            # Construct P2 by swapping sides between prev_state and state
            P2_prev_state = f"S_{P1_prev_skill}"
            P2_state = f"R_{P1_skill}"
            
            # Check if P2 exists in the transition matrix
            if P2_prev_state in sym_transition_matrix.index and P2_state in sym_transition_matrix.columns:
                # Calculate symmetric average for condition 2
                prob1 = sym_transition_matrix.at[P1_prev_state, P1_state]
                prob2 = sym_transition_matrix.at[P2_prev_state, P2_state]
                avg_prob = (prob1 + prob2) / 2

                # Update the transition probabilities with the average
                sym_transition_matrix.at[P1_prev_state, P1_state] = avg_prob
                sym_transition_matrix.at[P2_prev_state, P2_state] = avg_prob

                # Mark this pair as processed
                processed_pairs.add((P1_prev_state, P1_state))
                processed_pairs.add((P2_prev_state, P2_state))


In [41]:
def standardize_row(row):
    row_sum = row.sum()
    if row_sum != 0:
        return row / row_sum
    else:
        return row  # If the sum is 0, return the row unchanged

# Apply the standardization function to all rows of the transition matrix
sym_transition_matrix = sym_transition_matrix.apply(standardize_row, axis=1)

In [45]:
sym_transition_matrix

short_state,R_ACB,R_ACD,R_ACF,R_ANA,R_APP,R_AV0,R_AV5,R_AV6,R_AV8,R_AVB,...,S_F/,S_F=,S_P,S_R-,S_R/,S_R=,S_S#,S_S-,S_S=,S_SV
prev_state,,,,,,,,,,,,,,,,,,,,,
R_ACB,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.086022,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
R_ACD,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.145688,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
R_ACF,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.089984,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
R_ANA,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000090,0.000162,0.077989,0.000090,0.000000,0.000000,0.000252,0.000000,0.0,0.0
R_APP,0.0,0.0,0.0,0.000317,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000317,0.000271,0.000000,0.000000,0.000000,0.000271,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S_R=,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
S_S#,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
S_S-,0.0,0.0,0.0,0.093147,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001449,0.033696,0.011557,0.000000,0.000130,0.000000,0.000409,0.001059,0.0,0.0


In [46]:
sym_transition_matrix.to_csv('./data/sym_transition_matrix.csv')

#### Absorption Probabilities

In [47]:
def calculate_absorption_probabilities(transition_matrix, absorbing_states):
    # transient and absorbing states
    all_states = list(transition_matrix.index)
    transient_states = [state for state in all_states if state not in absorbing_states]

    # Extract submatrices Q and R
    Q = transition_matrix.loc[transient_states, transient_states].values
    R = transition_matrix.loc[transient_states, absorbing_states].values

    # Calculate N = (I - Q)^(-1)
    I = np.eye(len(Q))
    N = np.linalg.inv(I - Q)

    # Calculate absorption probabilities B = N * R
    B = np.dot(N, R)

    # Convert results to a DataFrame
    absorption_probabilities = pd.DataFrame(B, index=transient_states, columns=absorbing_states)
    
    return absorption_probabilities, N

In [48]:
absorbing_states = ["R_P", "S_P"]
absorption_probabilities, N = calculate_absorption_probabilities(transition_matrix, absorbing_states)

In [49]:
sym_absorption_probabilities, sym_N = calculate_absorption_probabilities(sym_transition_matrix, absorbing_states)

In [51]:
absorption_probabilities.to_excel('./data/absorption_prob_short.xlsx')
sym_absorption_probabilities.to_excel('./data/sym_absorption_prob_short.xlsx')

#### Contact Info and CWPA

CWPA - contact team win probability added

In [76]:
state_cols = ['match_id', 'point_id', 'skill', 'serving_team', 'player_name', 'player_id', 
              'team', 'side', 'point', 'prev_state', 'short_state']
# Keep useful columns and rename them
output_df = state_df[state_cols]
output_df.rename(columns={'short_state': 'state', 
                          'team': 'contact_team', 
                          'side': 'side_of_contact'}, inplace=True)
output_df.dropna(subset=['player_name'], inplace=True)
output_df.head(5)

,match_id,point_id,skill,serving_team,player_name,player_id,contact_team,side_of_contact,point,prev_state,state
1,5d7a5827529932230e12cc7d0fcbee40,0,Serve,University of Iowa,Addie VanderWeide,-334910.0,University of Iowa,S,False,NaN,S_SV
2,5d7a5827529932230e12cc7d0fcbee40,0,Reception,University of Iowa,Gabby Gonzales,-278794.0,Ohio State University,R,False,S_SV,R_R+
3,5d7a5827529932230e12cc7d0fcbee40,0,Set,University of Iowa,Mac Podraza,-262192.0,Ohio State University,R,False,R_R+,R_S#
4,5d7a5827529932230e12cc7d0fcbee40,0,Attack,University of Iowa,Emily Londot,-308957.0,Ohio State University,R,False,R_S#,R_AX6
5,5d7a5827529932230e12cc7d0fcbee40,0,Dig,University of Iowa,Lily Tessier,-381591.0,University of Iowa,S,False,R_AX6,S_D=


In [77]:
# create prob_dict with R_P absorption probabilities
prob_dict = sym_absorption_probabilities['R_P'].to_dict()

In [78]:
output_df.head(20)

,match_id,point_id,skill,serving_team,player_name,player_id,contact_team,side_of_contact,point,prev_state,state
1,5d7a5827529932230e12cc7d0fcbee40,0,Serve,University of Iowa,Addie VanderWeide,-334910.0,University of Iowa,S,False,NaN,S_SV
2,5d7a5827529932230e12cc7d0fcbee40,0,Reception,University of Iowa,Gabby Gonzales,-278794.0,Ohio State University,R,False,S_SV,R_R+
3,5d7a5827529932230e12cc7d0fcbee40,0,Set,University of Iowa,Mac Podraza,-262192.0,Ohio State University,R,False,R_R+,R_S#
4,5d7a5827529932230e12cc7d0fcbee40,0,Attack,University of Iowa,Emily Londot,-308957.0,Ohio State University,R,False,R_S#,R_AX6
5,5d7a5827529932230e12cc7d0fcbee40,0,Dig,University of Iowa,Lily Tessier,-381591.0,University of Iowa,S,False,R_AX6,S_D=
8,5d7a5827529932230e12cc7d0fcbee40,1,Serve,Ohio State University,Gabby Gonzales,-278794.0,Ohio State University,S,False,None,S_SV
9,5d7a5827529932230e12cc7d0fcbee40,1,Reception,Ohio State University,Addie VanderWeide,-334910.0,University of Iowa,R,False,S_SV,R_R+
10,5d7a5827529932230e12cc7d0fcbee40,1,Set,Ohio State University,Lily Tessier,-381591.0,University of Iowa,R,False,R_R+,R_S#
11,5d7a5827529932230e12cc7d0fcbee40,1,Attack,Ohio State University,Michelle Urquhart,-368212.0,University of Iowa,R,False,R_S#,R_AX5
12,5d7a5827529932230e12cc7d0fcbee40,1,Dig,Ohio State University,Kylie Murr,-278795.0,Ohio State University,S,False,R_AX5,S_D=


In [89]:
pd.isna(output_df.iloc[5]['prev_state'])

True

In [90]:
# create prob_dict with R_P absorption probabilities
prob_dict = sym_absorption_probabilities['R_P'].to_dict()

def state_win_prob(row):
    # If the contact side is R, return the absorption prob of "R_P"
    # If the contact side is S, return the absorption prob of "S_P"
    if row['side_of_contact'].startswith('R'):
        return prob_dict[row['state']]
    else:
        return 1 - prob_dict[row['state']]
    
def prev_state_win_prob(row):
    # If the contact side is R, return the absorption prob of "R_P"
    # If the contact side is S, return the absorption prob of "S_P"
    if pd.isna(row['prev_state']):
        return None
    if row['side_of_contact'] == 'R':
        return prob_dict[row['prev_state']]
    else:
        return 1 - prob_dict[row['prev_state']]

In [91]:
output_df['state_win_prob'] = output_df.apply(state_win_prob, axis=1)
output_df['prev_state_win_prob'] = output_df.apply(prev_state_win_prob, axis=1)

1         0.383155
2         0.580282
3         0.584979
4         0.616014
5         0.000000
            ...   
953850    0.383155
953851    0.577755
953852    0.584979
953853    0.600681
953854    0.000000
Length: 735574, dtype: float64

In [71]:
output_df.iloc[0]['state'].startswith('R')

False

In [34]:
state_df.to_csv('./data/point_state_data.csv', index=False)
state_df.to_csv('./data/point_state_data_semicolon.csv', sep=';', index=False)